In [ ]:
using Revise
using InstrumentControl
using InstrumentControl:E5071C, GS200, SMB100A, AWGM320XA, DigitizerM3102A 
using AxisArrays
using KeysightInstruments
using KeysightQubits
using JLD, FileIO, LsqFit
import Plots
Plots.plotlyjs()

import InstrumentControl: Stimulus, Response, Instrument, source, measure, PropertyStimulus

dB(x)= 20*log10(abs(x)) #to be used with VNA S params output
phas(x)=atan2(imag(x),real(x))

# Equipment Initialization

### Yokogawa GS200 DC Source

In [ ]:
yoko = InsGS200(tcpip_socket("169.254.65.200",7655))  #initializing instrument object
Vz = RampStim(yoko) 

### R&S SMB100A Signal Generators

In [ ]:
#iinitializing signal generator instrument objects
sigRead = InsSMB100A(tcpip_instr("169.254.1.20")) 
sigXY = InsSMB100A(tcpip_instr("169.254.235.224"))

In [ ]:
#defining stimuli for changing signal generator power and frequencies
xyPowerStim = PropertyStimulus(sigXY, PowerLevel, axisname=:xypower, axislabel="XY power (dBm)")
xyFreqStim = PropertyStimulus(sigXY, Frequency, axisname=:xyf, axislabel="XY frequency (GHz)");
readFreqStim = PropertyStimulus(sigRead, Frequency, axisname=:readf, axislabel="Readout frequency (GHz)");
readPowerStim = PropertyStimulus(sigRead, PowerLevel, axisname=:readoutpower, axislabel="Readout power (dBm)")

### E5071C Vector Network Analyzer(VNA)

In [ ]:
vna = InsE5071C(tcpip_socket("169.254.61.208",5025)) #initializing signal generator instrument object
readavailable(vna)
#NOTE!: THE LAST COMMAND IS SUPPOSED TO GIVE AN VI_ERROR_TMO; THAT IS JUST A BUG

In [ ]:
S_ParamsResponse = VNA.FrequencySweep(vna) #redefinition for convenience; this is a response object which measures S parameters 
VNAPowerStim = PropertyStimulus(vna, PowerLevel, axisname=:vnapower, axislabel="VNA power (dBm)")

# Finding Readout Resonator and Qubit Frequencies (Frequency Domain)

In [ ]:
#configuring VNA settings to measure S21 
vna[TriggerSource] = :InternalTrigger
vna[NumPoints] = 1001
vna[IFBandwidth] = 2000.
vna[Smoothing] = false
vna[Averaging] = true
vna[AveragingFactor] = 1
vna[VNA.Parameter] = :S21
vna[Timeout]=10000

In [ ]:
#imaging VNA S21 measurement at frequency range of interest --> Looking for resonator lorentzian
vna[PowerLevel] = -55
vna[FrequencyStart] = 6.883e9
vna[FrequencyStop] = 6.89e9
measure(S_ParamsResponse)
screen(vna)

In [ ]:
#Saturating resonator to see transition from |0> dispersed dressed resonance to bare cavity resonance
vna[FrequencyStart] = 6.875e9
vna[FrequencyStop] = 6.89e9

sweep(S_ParamsResponse, (VNAPowerStim, -60:5:-10))

In [ ]:
r = result(1)[:S21] #fill in job number here
Plots.heatmap(axisvalues(r)[2], axisvalues(r)[1]/1e9, dB.(r), xl="Readout power (dBm)", yl="VNA frequency (GHz)")

#### |0> dispersed frequency:
#### bare cavity frequency: 
#### χ = 

### Finding SQUID Frequency Period and Zero Flux Offset, in Volts 

In [ ]:
#Tuning qubit from max resonance to zero frequency with Z-line.If ω_r> ω_q_max, then as V_z is increased until it's... 
# half period value --> the qubit goes to zero frequency --> the detuning increases --> the dispersive shift decreases --> 
# readout resonance changes
# If ω_r < ω_q_max, then you will see an avoided level crossing 

vna[PowerLevel] = -60
vna[FrequencyStart] = 6.875e9
vna[FrequencyStop] = 6.89e9

sweep(S_ParamsResponse, (Vz, -0.8:0.025:0.8))

In [ ]:
r = result(2)[:S21]
Plots.heatmap(axisvalues(r)[2], axisvalues(r)[1]/1e9, dB.(r), xl="Voltage Bias (V)", yl="VNA frequency (GHz)")

#### V_offset = 
#### V_period = 

### Finding Qubit Frequency

In [ ]:
# measuring S21, in the vicinity of the readout resonator |0> dispersed frequency, while sweeping XY frequency with the... 
#... signal generator. If sigXY[Frequency] = qubit frequency, the qubit is Rabi oscillating so fast compared to Ω that... 
#... the resonator only sees an average of the two qubit states, i.e. motional averaging
vna[PowerLevel] = -60
vna[FrequencyStart] = 6.880e9
vna[FrequencyStop] = 6.888e9
source(V_z, 0.025) #source voltage offset so qubit is at maximum frequency

sweep(S_ParamsResponse, (xyFreqStim, 4.2e9:1e6:5.2e9), (xyPowerStim, [-20, -30, -40, -50]))

In [ ]:
# -20dBm power
r = result(6058)[Axis{:sparam}(:S21), Axis{:xypower}(1)]
Plots.heatmap(axisvalues(r)[2]/1e9, axisvalues(r)[1]/1e9, dB.(r), xl="XY frequency (GHz)", yl="VNA frequency (GHz)")

In [ ]:
#-30dBm Power
r = result(6057)[Axis{:sparam}(:S21), Axis{:xypower}(2)]
Plots.heatmap(axisvalues(r)[2]/1e9, axisvalues(r)[1]/1e9, dB.(r), xl="XY frequency (GHz)", yl="VNA frequency (GHz)")

In [ ]:
#-40dBm Power
r = result(6057)[Axis{:sparam}(:S21), Axis{:xypower}(3)]
Plots.heatmap(axisvalues(r)[2]/1e9, axisvalues(r)[1]/1e9, dB.(r), xl="XY frequency (GHz)", yl="VNA frequency (GHz)")

In [ ]:
# -50dBm power
r = result(6057)[Axis{:sparam}(:S21), Axis{:xypower}(4)]
Plots.heatmap(axisvalues(r)[2]/1e9, axisvalues(r)[1]/1e9, dB.(r), xl="XY frequency (GHz)", yl="VNA frequency (GHz)")